## 2.5: Advanced Geospatial Plotting

In [3]:
# Import libraries

import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [4]:
# Upload data

df = pd.read_csv('/Users/katiedallarosa/Desktop/CitiBike/nycitibike_data.csv', index_col = 0)

/var/folders/zp/dgvf82dj1f11lqylj8ps_3c00000gn/T/ipykernel_10945/849508513.py:3: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/katiedallarosa/Desktop/CitiBike/nycitibike_data.csv', index_col = 0)


#### In the data set from the previous task, create a new column with the value of 1. Then create a new aggregated dataframe that contains 3 columns: starting station, ending station, and the count of trips between those stations.

In [6]:
df.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,value,bike_rides_daily,merge_flag
0,1832,69198402DE2F09D9,electric_bike,2022-01-01 18:47:29.261,2022-01-01 19:34:01.835,Mott St & Prince St,5561.04,E 138 St & Cypress Ave,7669.1,40.723180,-73.994800,40.805559,-73.914718,member,2022-01-01,2.0,1,5125,both
1,2264,A88286580E006A23,classic_bike,2022-01-01 12:11:43.270,2022-01-01 12:22:41.986,Dock 72 Way & Market St,4804.02,Washington Ave & Greene Ave,4419.03,40.699850,-73.971410,40.686501,-73.965633,member,2022-01-01,2.0,1,5125,both
2,2505,5C8D80FADFCE6A5A,classic_bike,2022-01-01 12:39:49.213,2022-01-01 12:45:54.864,Great Jones St,5636.11,W 13 St & 7 Ave,6030.04,40.727434,-73.993790,40.737815,-73.999947,casual,2022-01-01,2.0,1,5125,both
3,3470,EE8147050ACEB21C,classic_bike,2022-01-01 10:44:01.137,2022-01-01 10:46:53.936,E 13 St & 2 Ave,5820.08,Cooper Square & Astor Pl,5712.03,40.731539,-73.985302,40.729515,-73.990753,member,2022-01-01,2.0,1,5125,both
4,3738,21B74269FD2FCDBD,classic_bike,2022-01-01 01:11:09.098,2022-01-01 01:36:58.774,W 17 St & 8 Ave,6148.02,Duane St & Hudson St,5359.12,40.741776,-74.001497,40.717030,-74.009250,casual,2022-01-01,2.0,1,5125,both


In [7]:
# Drop value column for next step

df = df.drop('value', axis=1)

In [8]:
# Create a value column and group by start and end station 

df['value'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()

In [9]:
df_group

,start_station_name,end_station_name,value
0,1 Ave & E 110 St,1 Ave & E 110 St,195
1,1 Ave & E 110 St,1 Ave & E 18 St,1
2,1 Ave & E 110 St,1 Ave & E 30 St,2
3,1 Ave & E 110 St,1 Ave & E 44 St,1
4,1 Ave & E 110 St,1 Ave & E 62 St,7
...,...,...,...
663988,Yankee Ferry Terminal,South St & Gouverneur Ln,3
663989,Yankee Ferry Terminal,South St & Whitehall St,27
663990,Yankee Ferry Terminal,W 15 St & 10 Ave,1
663991,Yankee Ferry Terminal,West St & Chambers St,2


In [10]:
df_latlong = df[['start_station_name','end_station_name','start_lat','start_lng','end_lat','end_lng']]
df_latlong = df_latlong.drop_duplicates(subset=['start_station_name','end_station_name'])

In [41]:
df_latlong.shape

(665601, 6)

In [43]:
df_group.shape

(663993, 3)

In [45]:
df_group = df_group.merge(df_latlong,on=['start_station_name','end_station_name'])

In [49]:
df_group

,start_station_name,end_station_name,trips,start_lat,start_lng,end_lat,end_lng
0,1 Ave & E 110 St,1 Ave & E 110 St,195,40.792327,-73.938300,40.792327,-73.938300
1,1 Ave & E 110 St,1 Ave & E 18 St,1,40.792327,-73.938300,40.733812,-73.980544
2,1 Ave & E 110 St,1 Ave & E 30 St,2,40.792327,-73.938300,40.741444,-73.975361
3,1 Ave & E 110 St,1 Ave & E 44 St,1,40.792327,-73.938300,40.750020,-73.969053
4,1 Ave & E 110 St,1 Ave & E 62 St,7,40.792327,-73.938300,40.761227,-73.960940
...,...,...,...,...,...,...,...
663988,Yankee Ferry Terminal,South St & Gouverneur Ln,3,40.687066,-74.016756,40.703554,-74.006702
663989,Yankee Ferry Terminal,South St & Whitehall St,27,40.687066,-74.016756,40.701221,-74.012342
663990,Yankee Ferry Terminal,W 15 St & 10 Ave,1,40.687066,-74.016756,40.742754,-74.007474
663991,Yankee Ferry Terminal,West St & Chambers St,2,40.687066,-74.016756,40.717548,-74.013221


In [53]:
df_plot = df_group.nlargest(n=100,columns='trips').reset_index(drop=True)
df_plot

,start_station_name,end_station_name,trips,start_lat,start_lng,end_lat,end_lng
0,Central Park S & 6 Ave,Central Park S & 6 Ave,3013,40.765909,-73.976342,40.765909,-73.976342
1,7 Ave & Central Park South,7 Ave & Central Park South,2088,40.766741,-73.979069,40.766741,-73.979069
2,Roosevelt Island Tramway,Roosevelt Island Tramway,2064,40.757284,-73.953600,40.757284,-73.953600
3,Grand Army Plaza & Central Park S,Grand Army Plaza & Central Park S,1820,40.764397,-73.973715,40.764397,-73.973715
4,Soissons Landing,Soissons Landing,1800,40.692317,-74.014866,40.692317,-74.014866
...,...,...,...,...,...,...,...
95,Central Park West & W 85 St,Central Park West & W 85 St,700,40.784760,-73.969862,40.784760,-73.969862
96,S 5 Pl & S 5 St,S 5 Pl & S 5 St,698,40.710451,-73.960876,40.710451,-73.960876
97,North Moore St & Greenwich St,Vesey Pl & River Terrace,696,40.720195,-74.010301,40.715338,-74.016584
98,46 Ave & 5 St,45 Ave & 21 St,695,40.747310,-73.954510,40.747371,-73.947773


In [61]:
df_plot.to_csv('df_plot.csv')

In [11]:
# Compare sum values of the value column and number of rows to ensure groupby funciton was done correctly

print(df_group['value'].sum())
print(df.shape)

7442145
(7459541, 19)


In [12]:
df_group.rename(columns = {'value': 'trips'}, inplace = True)

In [64]:
# Check statistics

df_group['trips'].describe()

count    663993.000000
mean         11.208168
std          30.078687
min           1.000000
25%           1.000000
50%           3.000000
75%           9.000000
max        3013.000000
Name: trips, dtype: float64

In [17]:
df_group['trips'].sort_index().value_counts()

trips
1       228699
2        94382
3        54465
4        36823
5        27149
         ...  
591          1
611          1
548          1
520          1
1353         1
Name: count, Length: 673, dtype: int64

In [18]:
df_group.to_csv('df_groupby_final.csv')

In [19]:
# Check data

df_group.head()

,start_station_name,end_station_name,trips
0,1 Ave & E 110 St,1 Ave & E 110 St,195
1,1 Ave & E 110 St,1 Ave & E 18 St,1
2,1 Ave & E 110 St,1 Ave & E 30 St,2
3,1 Ave & E 110 St,1 Ave & E 44 St,1
4,1 Ave & E 110 St,1 Ave & E 62 St,7
